In [2]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress


In [3]:
#set file paths and read in files
filepath = 'raw_data/MN_walkability.csv'
statepath = 'raw_data/state_and_county_fips_master.csv'

minnesota = pd.read_csv(filepath)
states = pd.read_csv(statepath)



In [4]:

#add leading 0's for counties
minnesota['COUNTYFP'] = minnesota['COUNTYFP'].astype(str)
minnesota['STATEFP'] = minnesota['STATEFP'].astype(str)
minnesota['COUNTYFP'] = minnesota['COUNTYFP'].str.zfill(3)

#concatenate state and county codes
minnesota['fips'] = minnesota['STATEFP'] + minnesota['COUNTYFP']
minnesota['fips'] = minnesota['fips'].astype('int64')
minnesota.head(10)

,Unnamed: 0,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,...,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,fips
0,125050,125051,2.716310e+11,2.716310e+11,27,163,71003,1,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,18,15,8,1,8.500000,12371.721880,4.274573e+06,27163
1,125051,125052,2.716310e+11,2.716310e+11,27,163,71003,2,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,18,16,7,1,8.333333,7746.443532,2.872370e+06,27163
2,125052,125053,2.716310e+11,2.716310e+11,27,163,71003,3,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,15,6,9,1,6.833333,9884.396703,2.854760e+06,27163
3,125053,125054,2.716310e+11,2.716310e+11,27,163,71006,1,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,20,20,20,1,13.666667,7814.948545,3.005446e+06,27163
4,125054,125055,2.716310e+11,2.716310e+11,27,163,71006,2,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,20,18,18,1,12.666667,10906.431880,3.546875e+06,27163
5,125055,125056,2.713110e+11,2.713110e+11,27,131,70800,5,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,11,5,16,1,8.333333,3945.373100,4.936691e+05,27131
6,125056,125057,2.714500e+11,2.714500e+11,27,145,601,1,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,15,20,18,1,12.166667,3645.290542,5.845590e+05,27145
7,125057,125058,2.703710e+11,2.703710e+11,27,037,60904,2,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,20,20,12,1,11.000000,7146.198431,2.405088e+06,27037
8,125058,125059,2.703710e+11,2.703710e+11,27,037,60604,2,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,7,18,10,1,7.833333,7165.978041,2.552602e+06,27037
9,125059,125060,2.703710e+11,2.703710e+11,27,037,60817,4,378.0,"Minneapolis-St. Paul, MN-WI",...,-99999.0,-99999.0,4,11,12,1,6.833333,8984.860085,4.239370e+06,27037


In [17]:
#merge main dataset with fips codes dataset
merged_df = pd.merge(minnesota,states,on='fips')

#Choose with columns you want to view (feel free to add more!)
reduced_columns = merged_df[['GEOID20','CSA_Name','name','state','TotPop','NatWalkInd']]
reduced_columns = reduced_columns.rename(columns={'GEOID20':'Block Group','CSA_Name':'Metro Area','name':'County','TotPop':'Population','NatWalkInd':'Walking Index'})
reduced_columns

,Block Group,Metro Area,County,state,Population,Walking Index
0,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,1934,8.500000
1,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,574,8.333333
2,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,998,6.833333
3,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,2506,13.666667
4,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,2350,12.666667
...,...,...,...,...,...,...
4106,2.704000e+11,"Rochester-Austin, MN",Dodge County,MN,1785,15.166667
4107,2.704000e+11,"Rochester-Austin, MN",Dodge County,MN,1294,13.000000
4108,2.704000e+11,"Rochester-Austin, MN",Dodge County,MN,1757,15.833333
4109,2.704000e+11,"Rochester-Austin, MN",Dodge County,MN,974,7.500000


In [15]:
#add buckets for walkability

for index,row in reduced_columns.iterrows():
    if reduced_columns.loc[index, "NatWalkInd"] <= 5.75:
        reduced_columns.loc[index,'Walkability_bucket'] = 'Least Walkable'
    elif reduced_columns.loc[index, "NatWalkInd"] <= 10.5:
        reduced_columns.loc[index,'Walkability_bucket'] ='Below Average Walkability'
    elif reduced_columns.loc[index, "NatWalkInd"] <= 15.25:
        reduced_columns.loc[index,'Walkability_bucket'] = 'Above Average Walkability'
    elif reduced_columns.loc[index, "NatWalkInd"] > 15.25:
        reduced_columns.loc[index,'Walkability_bucket'] = 'Most Walkable'
reduced_columns.head(20)  

/var/folders/06/cl4dcprx4mgfy2x504fmy4lw0000gn/T/ipykernel_72472/577720634.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_columns.loc[index,'Walkability_bucket'] ='Below Average Walkability'


,GEOID20,CSA_Name,name,state,TotPop,NatWalkInd,Walkability_bucket
0,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,1934,8.500000,Below Average Walkability
1,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,574,8.333333,Below Average Walkability
2,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,998,6.833333,Below Average Walkability
3,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,2506,13.666667,Above Average Walkability
4,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,2350,12.666667,Above Average Walkability
5,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,953,9.500000,Below Average Walkability
6,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,860,5.166667,Least Walkable
7,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,1918,5.333333,Least Walkable
8,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,1470,12.666667,Above Average Walkability
9,2.716310e+11,"Minneapolis-St. Paul, MN-WI",Washington County,MN,1016,10.500000,Below Average Walkability
